# 3W - Strategy Thinking project

In [1]:
# IMPORTS AND CONFIGURATIONS

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
# from matplotlib.ticker import FuncFormatter
# import plotly.express as px
%matplotlib inline

import pandas as pd
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler
# from imblearn.over_sampling import SMOTE
# from imblearn.combine import SMOTETomek

import glob
import os
import time

from sklearn.dummy import DummyClassifier
import sklearn.metrics as metrics

# importing classes and functions for PCA, hyperparametrisation and cross-validation
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score



## Data Preparation

Preprocessing a dataset through data characterisation involves summarising the features and characteristics present in the data using statistical measures and visualisations techniques such as bar charts and scatter plots. After this stage, it should be possible to identify biases, patterns, trends, and any missing or irrelevant data in the data set that may need to be addressed.

This dataset is composed by instances of eight types of undesirable events characterized by eight process variables from three different sources: real instances, simulated instances and hand-drawn instances. All real instances were taken from the plant information system that is used to monitor the industrial processes at an operational unit in Brazilian state of Espírito Santo. The simulated instances were all generated using OLGA ([Schlumberger](https://www.software.slb.com/products/olga)), a dynamic multiphase flow simulator that is widely used by oil companies worldwide (Andreolli, 2016). Finally, the hand-drawn instances were generated by a specific tool developed by Petrobras researchers for this dataset to incorporate undesirable events classfied as rare. 
 
### Data Characterisation
The data consists of over 50 million observations, with 13 columns of data for each observation. The first column, `label`, indicates the event type for each observation. The second column, `well`, contains the name of the well the observation was taken from. Hand-drawn and simulated instances have fixed names for in this column, while real instances have names masked with incremental id. The third column, `id`, is an identifier for the observation and it is incremental for hand-drawn and simulated instances, while each real instance has an id generated from its first timestamp. The columns representing the process variables are:

* **P-PDG**: pressure variable at the Permanent Downhole Gauge (PDG) - installed on Christmas Tree;
* **P-TPT**: pressure variable at the Temperature and Pressure Transducer (TPT) - installed on Christmas Tree;
* **T-TPT**: temperature variable at the Temperature and Pressure Transducer (TPT);
* **P-MON-CKP**: pressure variable upstream of the production choke (CKP) - located on platform;
* **T-JUS-CKP**: temperature variable downstream of the production choke (CKP);
* **P-JUS-CKGL**: pressure variable upstream of the gas lift choke (CKGL);
* **T-JUS-CKGL**: temperature variable upstream of the gas lift choke (CKGL);
* **QGL**: gas lift flow rate;

The pressure features are measured in Pascal (Pa), the volumetric flow rate features are measured in standard cubic meters per second (SCM/s), and the temperature features are measured in degrees Celsius (°C).

Other information are also loaded into each pandas Dataframe:

* **label**: instance label (event type) - target variable;
* **well**: well name. Hand-drawn and simulated instances have fixed names (respectively, `drawn` and `simulated`. Real instances have names masked with incremental id;
* **id**: instance identifier. Hand-drawn and simulated instances have incremental id. Each real instance has an id generated from its first timestamp;
* **class**: Although it can be used to identify periods of normal operation, fault transients, and faulty steady states, which can help with diagnosis and maintenance, it is a category which results from label, which is our target here

The labels are:
* 0 - Normal Operation = `Normal`
* 1 - Abrupt Increase of BSW = `AbrIncrBSW`
* 2 - Spurious Closure of DHSV = `SpurClosDHSW`
* 3 - Severe Slugging = `SevSlug`
* 4 - Flow Instability = `FlowInst`
* 5 - Rapid Productivity Loss = `RProdLoss`
* 6 - Quick Restriction in PCK	= `QuiRestrPCK`
* 7 - Scaling in PCK = `ScalingPCK`
* 8 - Hydrate in Production Line = `HydrProdLine`

More information about these variables can be obtained from the following publicly available documents:

* ***Option in Portuguese***: R.E.V. Vargas. Base de dados e benchmarks para prognóstico de anomalias em sistemas de elevação de petróleo. Universidade Federal do Espírito Santo. Doctoral thesis. 2019. https://github.com/petrobras/3W/raw/master/docs/doctoral_thesis_ricardo_vargas.pdf.
* ***Option in English***: B.G. Carvalho. Evaluating machine learning techniques for detection of flow instability events in offshore oil wells. Universidade Federal do Espírito Santo. Master's degree dissertation. 2021. https://github.com/petrobras/3W/raw/master/docs/master_degree_dissertation_bruno_carvalho.pdf.

In order to maintain the realistic aspects of the data, the dataset was extracted without preprocessing, including the presence of `NaN` values, frozen variables due to sensor or communication issues, instances with varying sizes, and outliers (R.E.V. Vargas, et al. 2019). 

From all 50,822,124 entries, 3,086,851 are duplicated, that is, approximately 6.07% of total. These duplicated rows may be related to frozen variables from real instances, as simulated and hand-drawn instances are naturally free of such problems. Although no missing values were found for columns `label`, `well`, and `id`, other features presented null or absent values. Notably, the column `T-JUS-CKGL` turned out to be completely empty.

In [2]:
df = pd.read_csv('3Wdataset_real.csv', index_col=None, parse_dates=['timestamp'])

# only using real, since simulated and drawn doesnt have QGL and P-JUS-CKGL
# df_real.info()

# df_drawn = pd.read_csv('3Wdataset_drawn.csv', index_col=None)
# # df_drawn.info()

# df_sim1 = pd.read_csv('3Wdataset_simulated_1of2.csv', index_col=None)
# # df_sim1.info()

# df_sim2 = pd.read_csv('3Wdataset_simulated_2of2.csv', index_col=None)
# # df_sim2.info()

# df = pd.concat([
#     df_real,
#     df_sim1,
#     df_sim2,
#     df_drawn
# ])

# df = df.drop('source', axis=1)

# dismissing temporary DFs to release memory
# del df_sim1
# del df_sim2
# del df_real
# del df_drawn

df.shape

(13952911, 14)

### Exploratory Data Analysis

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
from dataprep.eda import create_report
report = create_report(df, title='My Report')

In [ ]:
# Finding missing values
missing = df.isnull()
missing.sum()

total_missing = df.isnull().sum()
percent_missing = total_missing * 100 / len(df)
missing_value_df = pd.DataFrame({'percent': percent_missing, 'total':total_missing})

missing_value_df

### Data Cleaning

In [ ]:
# drop rows with missing or null class column
df_clean = df.dropna(subset=['P-PDG','P-TPT','T-JUS-CKP','P-MON-CKP','T-TPT','P-MON-CKP','QGL','P-JUS-CKGL'])

# first interaction will dismiss timestamp in order to remove duplicates, even if it overlooks frozen
# variables due to sensor or communication issues

# removing class, since it results from label
# removing ID from df

df_clean = df_clean.drop([
#     'timestamp', 
    'class',
    'T-JUS-CKGL', # T-JUS-CKGL is empty
    'id', 
    'source'
], axis=1)

# checking duplicated rows after removing ids
df_clean = df_clean.drop_duplicates()

df_clean.info()

In [ ]:
# Finding missing values
missing = df_clean.isnull()
missing.sum()

total_missing = df_clean.isnull().sum()
percent_missing = total_missing * 100 / len(df_clean)
missing_value_df = pd.DataFrame({'percent': percent_missing, 'total':total_missing})

# Total of Missing Values
missing_value_df.sum()

In [ ]:
df_clean.describe()

In [ ]:
# Before Scaling, with Outliers

df_clean.plot(kind='box', figsize=(8, 5), showfliers=True)
plt.title('Boxplot of Features Before Scaling')
plt.xlabel('Feature')
plt.ylabel('Value')
ax = plt.gca()
ax.ticklabel_format(style='plain', axis='y', useOffset=False)
plt.show()

### Feature Engineering

In [ ]:
dt_feat = df_clean

# Change 'label' column to object dtype
dt_feat['label'] = dt_feat['label'].astype('object') 

# Create boolean columns for each label
label_dummies = pd.get_dummies(dt_feat['label'], prefix='label')
dt_feat = pd.concat([dt_feat, label_dummies], axis=1)

# Rename boolean columns
column_names = {
    'label_0': 'Normal',
    'label_1': 'AbrIncrBSW',
    'label_2': 'SpurClosDHSW',
    'label_3': 'SevSlug',
    'label_4': 'FlowInst',
    'label_5': 'RProdLoss',
    'label_6': 'QuiRestrPCK',
    'label_7': 'ScalingPCK',
    'label_8': 'HydrProdLine'
}
dt_feat = dt_feat.rename(columns=column_names)

# Drop the original 'label' column and Normal column, since all other events must be 0
dt_feat = dt_feat.drop(['label','Normal'], axis=1)

dt_feat.info()

In [ ]:
dt_feat_target = dt_feat.drop([
    'AbrIncrBSW',
    'SpurClosDHSW',
#     'SevSlug',
    'FlowInst',
    'RProdLoss',
    'QuiRestrPCK',
    'ScalingPCK'
#     ,'HydrProdLine'
], axis=1)

dt_feat_target.info()

In [ ]:
# Computing the correlations
corr = dt_feat_target.corr()

# Creating a boolean mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Plotting the heatmap with the mask
fig, ax = plt.subplots(figsize=(8, 5))
sns.set(font_scale=0.6)
sns.heatmap(corr, annot=True, mask=mask, ax=ax, fmt='.2f')
plt.show()

### Handling Imbalanced Data

In [ ]:
# defining features (X) and label (y)

target = 'SevSlug'

X = dt_feat_target.drop([target,'timestamp','well'], axis=1)
y = dt_feat_target['SevSlug']

# splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# calculating the percentage of each label in the dataset
y_train.value_counts(normalize=True)

In [ ]:
# establishing a baseline with a DummyClassifier
dummyc = DummyClassifier()
dummyc.fit(X_train, y_train)

# retrieving score for DummyClassifier
score = dummyc.score(X_train, y_train)
y_predicted = dummyc.predict(X_test)

print("Score: ", score)
print("Accuracy: ",metrics.accuracy_score(y_test, y_predicted))

In [ ]:
X.isna().sum().sum()

In [ ]:
X.shape

In [ ]:
# balancing data 
balancing = RandomUnderSampler(random_state=42)
# balancing = SMOTE(random_state=42)
# balancing = SMOTETomek(random_state=42)

X_resampled, y_resampled = balancing.fit_resample(X, y)

In [ ]:
X_resampled.isna().sum().sum()

In [ ]:
X_resampled.shape

In [ ]:
# Finding missing values
missing = X_resampled.isnull()
missing.sum()

total_missing = X_resampled.isnull().sum()
percent_missing = total_missing * 100 / len(X_resampled)
missing_value_df = pd.DataFrame({'percent': percent_missing, 'total':total_missing})

missing_value_df.T

In [ ]:
# splitting the balanced datasets for training and test
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

# calculating the percentage of each label in the dataset after undersampling
y_train.value_counts(normalize=True)

### Data Scaling

In [ ]:
X_train.isna().sum()

In [ ]:
scaler = StandardScaler()
X_train_scaled_data = scaler.fit_transform(X_train)
X_train_scaled = pd.DataFrame(X_train_scaled_data, columns=X_train.columns)

X_train_scaled.info()

In [ ]:
# normalising test features
X_test_scaled_data = scaler.fit_transform(X_test)
X_test_scaled = pd.DataFrame(X_test_scaled_data, columns=X_test.columns)
X_test_scaled.info()

In [ ]:
X_train_scaled.isna().sum()

In [ ]:
X_train.shape

In [ ]:
# After Scaling

X_train_scaled.plot(kind='box', figsize=(8, 5))
plt.title('Boxplot of Features After Scaling')
plt.xlabel('Feature')
plt.ylabel('Value')
ax = plt.gca()
ax.ticklabel_format(style='plain', axis='y', useOffset=False)
plt.show()

## Dimensionality Reduction

In [ ]:
# initialising PCA with 3 components
pca = PCA(n_components=3)

# fitting PCA with features after undersampling and normalisation
pca.fit(X_train_scaled)

# reducing dimensionality in undersampled, normalised training dataset
X_train_pca = pca.transform(X_train_scaled)

print("Original shape: {}".format(str(X_train_scaled.shape)))
print("Reduced shape: {}".format(str(X_train_pca.shape)))

In [ ]:
# retrieving variance captured by the components
variance_captured = pca.explained_variance_ratio_
print(variance_captured)

In [ ]:
cum_sum_eigenvalues = np.cumsum(variance_captured)

plt.bar(range(0,len(variance_captured)), variance_captured, alpha=0.5, align='center', label='Individual explained variance')
plt.step(range(0,len(cum_sum_eigenvalues)), cum_sum_eigenvalues, where='mid',label='Cumulative explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal component index')
plt.legend(loc='best')
plt.tight_layout()
plt.show()

In [ ]:
fig = px.scatter_3d(X_train_pca, x=X_train_pca[:,0], y=X_train_pca[:,1], z=X_train_pca[:,2],
              color=y_train)
fig.show()

In [ ]:
# creating a matrix representing the directions of maximum variance in the data
plt.matshow(pca.components_, cmap='viridis')
plt.yticks([0, 1, 2], ["1st component", "2nd component", "3rd component"])
plt.colorbar()
plt.xticks(range(len(list(X_train_scaled.columns))), list(X_train_scaled.columns), rotation=60, ha='left')
plt.title('Figure 9 - Features x PCA Transformation')
plt.xlabel("Feature")
plt.ylabel("Principal components")

## Machine Learning

In [ ]:
# initialising PCA for test data, with 2 components
pca_test = PCA(n_components=3)

# fitting PCA with test features after normalisation
pca_test.fit(X_test_scaled)

# reducing dimensionality in normalised test dataset
X_test_pca = pca.transform(X_test_scaled)

print("Original shape: {}".format(str(X_test_scaled.shape)))
print("Reduced shape: {}".format(str(X_test_pca.shape)))

In [ ]:
# DummyClassifier x Normalised data
dummyc = DummyClassifier()
dummyc.fit(X_train_pca, y_train)

# confirming if score for Dummy classifier results from a balanced dataset
score = dummyc.score(X_train_pca, y_train)
y_predicted = dummyc.predict(X_test_pca)

print("Score: ", score)
print("Accuracy: ",metrics.accuracy_score(y_test, y_predicted))

### Linear SVC

#### Hyperparameter optimisation

In [ ]:
start_time = time.time()
print("Start time:", time.ctime(start_time))

from sklearn.svm import LinearSVC

# Dict containing different parameters for LinearSVC 
param_grid = {
    'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
    'penalty':['l1', 'l2'],
    'dual': [False, True]
}

# LinearSVC classifier
lin_clf = LinearSVC()

# GridSearchCV object is instantiated with these parameters and fit to training data
grid_search_lsvc = GridSearchCV(estimator=lin_clf, param_grid=param_grid, verbose=1)

# GridSearchCV object will be used to find the optimal combination of these parameters for the classification model
grid_search_lsvc.fit(X_train_pca, y_train)

end_time = time.time()
print("End time:", time.ctime(end_time))
os.system('say "Victor, the Hyperparameters for Linear SVC were successfully found."')

In [ ]:
# best accuracy achieved in the grid search and the best estimator
print("best accuracy", grid_search_lsvc.best_score_)
print(grid_search_lsvc.best_estimator_)

In [ ]:
# retrieving optimal values for the parameters dual, loss, multi_class, and penalty from grid search
optimal_dual = grid_search_lsvc.best_params_['dual']
# optimal_loss = grid_search_lsvc.best_params_['loss']
optimal_c = grid_search_lsvc.best_params_['C']
optimal_penalty = grid_search_lsvc.best_params_['penalty']

# instantiating a new LinearSVC classifier with optimal values
lin_clf = LinearSVC(dual=optimal_dual, C=optimal_c, penalty=optimal_penalty)
lin_clf.fit(X_train_pca, y_train)

# calculating score and accuracy
score = lin_clf.score(X_train_pca, y_train)
y_predicted_lin_clf = lin_clf.predict(X_test_pca)

print("Score: ", score)
print("Accuracy: ",metrics.accuracy_score(y_test, y_predicted_lin_clf))

### SGD Classifier

#### Hyperparameter optimisation

In [ ]:
start_time = time.time()
print("Start time:", time.ctime(start_time))

from sklearn.linear_model import SGDClassifier

# defining parameter grid for grid search
param_grid = {
    'max_iter': [1, 2, 5, 10, 20, 50, 100, 200, 500],
    'loss': [
        'hinge','log_loss','log','modified_huber','squared_hinge',
        'perceptron','squared_error','huber',
        'epsilon_insensitive','squared_epsilon_insensitive'
    ]
}

# instantiating SGD classifier
SGD = SGDClassifier()

# setting up grid search with SGD classifier and param_grid
grid_search_sgd = GridSearchCV(estimator=SGD, param_grid=param_grid, verbose=1)

# fitting grid search to training data
grid_search_sgd.fit(X_train_pca, y_train)

end_time = time.time()
print("End time:", time.ctime(end_time))
os.system('say "Victor, the Hyperparameters for SGD classifier were successfully found."')

In [ ]:
# best accuracy achieved in the grid search and the best estimator.
print("best accuracy", grid_search_sgd.best_score_)
print(grid_search_sgd.best_estimator_)

In [ ]:
optimal_max_iter = grid_search_sgd.best_params_['max_iter']
optimal_loss = grid_search_sgd.best_params_['loss']
# optimal_p = grid_search_knn.best_params_['p']

# a new KNeighborsClassifier object created and fitted with the training data
SGD = SGDClassifier(max_iter=optimal_max_iter, loss=optimal_loss)
SGD.fit(X_train_pca, y_train)

# calculating score and accuracy
score = SGD.score(X_train_pca, y_train)
y_predicted_sgd = SGD.predict(X_test_pca)

print("Score: ", score)
print("Accuracy: ",metrics.accuracy_score(y_test, y_predicted_sgd))

### k-Nearest Neighbours

#### Hyperparameter optimisation

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
start_time = time.time()
print("Start time:", time.ctime(start_time))

from sklearn.neighbors import KNeighborsClassifier

# defining parameter grid for grid search
param_grid = {
    'n_neighbors': range(3, 218, 5),
    'weights': ['uniform','distance']
#     'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute']
}

# instantiating kNN classifier
kNN = KNeighborsClassifier()

# setting up grid search with kNN classifier and param_grid
grid_search_knn = GridSearchCV(estimator=kNN, param_grid=param_grid, verbose=1)

# fitting grid search to training data
grid_search_knn.fit(X_train_pca, y_train)

end_time = time.time()
print("End time:", time.ctime(end_time))

os.system('say "Victor, the Hyperparameters for kNN were successfully found."')

In [ ]:
# best accuracy achieved in the grid search and the best estimator.
print("best accuracy", grid_search_knn.best_score_)
print(grid_search_knn.best_estimator_)

In [ ]:
# modeling after optimal k ("sweet spot") and weights values were determined
optimal_k = grid_search_knn.best_params_['n_neighbors']
optimal_weight = grid_search_knn.best_params_['weights']
# optimal_p = grid_search_knn.best_params_['p']

# a new KNeighborsClassifier object created and fitted with the training data
kNN = KNeighborsClassifier(n_neighbors=optimal_k, weights=optimal_weight)
kNN.fit(X_train_pca, y_train)

# calculating score and accuracy
score = kNN.score(X_train_pca, y_train)
y_predicted_knn = kNN.predict(X_test_pca)

print("Score: ", score)
print("Accuracy: ",metrics.accuracy_score(y_test, y_predicted_knn))

In [ ]:
start_time = time.time()
print("Start time:", time.ctime(start_time))

# initialising arrays for storing train and test accuracy
neighbors = np.arange(3, 218, 5)     
train_accuracy = np.zeros(len(neighbors))    
test_accuracy = np.zeros(len(neighbors))    

# looping over different values of k
for i, k in enumerate(neighbors):                         
    kNN = KNeighborsClassifier(n_neighbors=k, weights=optimal_weight)           
    kNN.fit(X_train_pca, y_train)                             
    train_accuracy[i] = kNN.score(X_train_pca, y_train)       
    test_accuracy[i] = kNN.score(X_test_pca, y_test)  
    
end_time = time.time()
print("End time:", time.ctime(end_time))

In [ ]:
# creating figure, adding title
plt.figure(figsize = (10, 6))
plt.title('Figure 10 - kNN accuracy with varying number of neighbors', fontsize = 10)

# plotting the test accuracy and traning accuracy x number of neighbours
plt.plot(neighbors, test_accuracy, label = 'Testing Accuracy')
plt.plot(neighbors, train_accuracy, label = 'Training accuracy')

# adding legend, axes labels, setting font size and axes ticks
plt.legend(prop={'size': 10})
plt.xlabel('Number of neighbors', fontsize = 10)
plt.ylabel('Accuracy', fontsize = 10)
plt.xticks(fontsize = 10)
plt.yticks(fontsize = 10)

plt.show()

## Conclusion

### Classification Report

In [ ]:
# generating a classification report for LinearSVC
cr_linearsvc = metrics.classification_report(y_test, y_predicted_lin_clf, digits=4)

# generating a classification report for SGD
cr_sgd = metrics.classification_report(y_test, y_predicted_sgd, digits=4)

# generating a classification report for kNN
cr_knn = metrics.classification_report(y_test, y_predicted_knn, digits=4)

#### Linear SVC

In [ ]:
# printing classification report for LinearSVC
print(cr_linearsvc)

#### SGD Classifier

In [ ]:
# printing classification report for SGD
print(cr_sgd)

#### k-Neighbours Classifier

In [ ]:
# printing classification report for kNN classifier
print(cr_knn)

### Confusion Matrix

In [ ]:
# setting labels valid for all following confusion matrices
cm_labels = ['No Sev Slug (0)', 'Sev Slug (1)']

#### Linear SVC

In [ ]:
# generating confusion matrix using the test labels and the predicted labels from LinearSVC classifier
cm = metrics.confusion_matrix(y_test, y_predicted_lin_clf)

# converting the confusion matrix into a pandas dataframe
cm_dataframe = pd.DataFrame(cm, index=cm_labels, columns=cm_labels)

# creating a heatmap using using seaborn
sns.heatmap(cm_dataframe, annot=True, fmt='d')

#### SGD Classifier

In [ ]:
# generating confusion matrix using the test labels and the predicted labels from LinearSVC classifier
cm = metrics.confusion_matrix(y_test, y_predicted_sgd)

# converting the confusion matrix into a pandas dataframe
cm_dataframe = pd.DataFrame(cm, index=cm_labels, columns=cm_labels)

# creating a heatmap using using seaborn
sns.heatmap(cm_dataframe, annot=True, fmt='d')

#### k-Neighbours Classifier

In [ ]:
# generating confusion matrix using the test labels and the predicted labels from kNeighbour Classifier
cm = metrics.confusion_matrix(y_test, y_predicted_knn)

# converting the confusion matrix into a pandas dataframe
cm_dataframe = pd.DataFrame(cm, index=cm_labels, columns=cm_labels)

# creating a heatmap using using seaborn
sns.heatmap(cm_dataframe, annot=True, fmt='d')

### Cross Validation

In [ ]:
# generating a 10-fold cross-validation for LinearSVC
scores = cross_val_score(lin_clf, X_train_pca, y_train, cv=10)

print(f"Mean score: {np.mean(scores):.3f}")
print(f"Standard deviation: {np.std(scores):.3f}")

In [ ]:
# generating a 10-fold cross-validation for SGD
scores = cross_val_score(SGD, X_train_pca, y_train, cv=10)

print(f"Mean score: {np.mean(scores):.3f}")
print(f"Standard deviation: {np.std(scores):.3f}")

In [ ]:
# generating a 10-fold cross validation for kNN
scores = cross_val_score(kNN, X_train_pca, y_train, cv=10)

print(f"Mean score: {np.mean(scores):.3f}")
print(f"Standard deviation: {np.std(scores):.3f}")

Ideas:
* remove outliers?
* other under sampling algorithms